In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd

import torch; torch.set_default_tensor_type(torch.cuda.FloatTensor)

hist_data = pd.read_csv('../datasets/hist_data.csv.gz')
test_data = pd.read_csv('../datasets/test.csv.gz')


In [5]:
from models.nnmodel import NNModel

In [6]:
nnm = NNModel()

In [7]:
nnm.train(hist_data, n_epochs=20)

Epoch:  0
Loss:  53280.81252002716
Epoch:  1


KeyboardInterrupt: 

In [ ]:
#nnm.model.to("cuda")
nnm.model.predict(444)

In [7]:
nnm.predict_item(203338264)

[202872237, 202872237, 202872237, 202820148, 202820148]

In [8]:
test = nnm.predict(test_data)
test['preds'].to_csv('pred.csv')

In [9]:
nnm.quality(test_data)

0.20449908850109738

In [11]:
test.preds.sample(100)

pav_order_id
98520039491    [203068900, 202872237, 202820148]
98520621894    [202820148, 203068900, 202872237]
98514372058               [202820148, 202872237]
98513786140    [202872237, 202820148, 203068900]
98518505603               [202820148, 202872237]
                             ...                
98518859048    [202872237, 202820148, 203068900]
98514515483    [202872237, 202820148, 203068900]
98520722782    [203068900, 202872237, 202820148]
98518057474               [202820148, 202872237]
98519411722               [202820148, 202872237]
Name: preds, Length: 100, dtype: object

In [2]:
# prep_input(hist_data)

In [3]:
import random


In [68]:

def sample1(src, seed, n_ctx=3, n_smpl=15):
    max_size = len(src)
    vals = [*src]
    samples = torch.zeros(n_smpl, seed + 1)
    for i in range(n_smpl):
        sym = random.randint(0, seed)
        rz = 1 if sym in src else 0
        j = random.randint(0, max_size - n_ctx)
        samples[i, vals[j:j + n_ctx]] = 1
        samples[i, seed] = rz
    return samples
    
sample1(set([1, 5, 6, 10, 1, 7]), 10, 3, 5)

tensor([[0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1.]])

In [14]:
def sample2(src, seed, n_smpl):
    max_size = len(src)
    vals = [*src]
    result = []
    for i in range(n_smpl):
        j = random.randint(1, max_size-1)
        context = vals[j-1]
        target = vals[j]
        result += [(target, context)]
    return result

In [15]:
prod_to_ix = {itemd_id: ix for ix, itemd_id in enumerate(hist_data["item_id"].unique()) }

def prep_input(X_train, seed=64):
    X_train["ix"] = X_train["item_id"].map(prod_to_ix)
    pairs = X_train[['pav_order_id', 'item_id']]\
                .groupby('pav_order_id')\
                .agg(set)
    x = pairs['item_id'].apply(lambda x: sample2(x, seed, n_smpl=5))
    return x
    


In [16]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [17]:
ngrams = prep_input(hist_data)

NameError: name 'random' is not defined

In [ ]:
ngrams = flatten([*ngrams.values])
ngrams[1:10]

In [74]:
from torch import nn
from torch.nn import functional as F

class SkipgramModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(SkipgramModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.linear2 = nn.Linear(128, context_size * vocab_size)
        #self.parameters['context_size'] = context_size

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))  # -1 implies size inferred for that index from the size of the data
        #print(np.mean(np.mean(self.linear2.weight.data.numpy())))
        out1 = F.relu(self.linear1(embeds)) # output of first layer
        out2 = self.linear2(out1)           # output of second layer
        #print(embeds)
        log_probs = F.log_softmax(out2, dim=1).view(CONTEXT_SIZE,-1)
        return log_probs

    def predict(self,input):
        context_idxs = torch.tensor([prod_to_ix[input]], dtype=torch.long)
        res = self.forward(context_idxs)
        res_arg = torch.argmax(res)
        res_val, res_ind = res.sort(descending=True)
        indices = [res_ind[i][0] for i in np.arange(0,3)]
        return indices


    def freeze_layer(self,layer):
        for name,child in model.named_children():
            print(name,child)
            if(name == layer):
                for names,params in child.named_parameters():
                    print(names,params)
                    print(params.size())
                    params.requires_grad= False

    def print_layer_parameters(self):
        for name,child in model.named_children():
                print(name,child)
                for names,params in child.named_parameters():
                    print(names,params)
                    print(params.size())

    def write_embedding_to_file(self,filename):
        for i in self.embeddings.parameters():
            weights = i.data.numpy()
        np.save(filename,weights)


In [76]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [77]:
model.to(device)

SkipgramModeler(
  (embeddings): Embedding(52472, 10)
  (linear1): Linear(in_features=10, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=157416, bias=True)
)

In [78]:
#context_idxs

In [79]:
losses = []

for epoch in range(50):
    total_loss = 0
    #------- Embedding layers are trained as well here ----#
    #lookup_tensor = torch.tensor([word_to_ix["poor"]], dtype=torch.long)
    #hello_embed = model.embeddings(lookup_tensor)
    #print(hello_embed)
    # -----------------------------------------------------#

    model.predict(203401622)
    print("Epoch: ", epoch)
    
    carts = hist_data.pav_order_id.drop_duplicates().sample(1000).values
    q = hist_data.pav_order_id.isin(carts)
    ngrams = flatten([*prep_input(hist_data[q]).values])
    
    for target, context  in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        #print(context,target)


        context_idxs = torch.tensor([prod_to_ix[context]], dtype=torch.long)
        context_idxs.to(device)
        #print("Context id",context_idxs)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
        #print(log_probs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        target_list = torch.tensor([prod_to_ix[target], prod_to_ix[target], prod_to_ix[target]], dtype=torch.long)
        loss = loss_function(log_probs, target_list)
        #print(loss)

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    print("Loss: ", total_loss)
    losses.append(total_loss)

0


/tmp/ipykernel_6309/818288104.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["ix"] = X_train["item_id"].map(prod_to_ix)


Loss:  59962.70633029938
1
Loss:  59742.66466999054
2
Loss:  59536.25030422211
3
Loss:  59329.45563411713
4
Loss:  59077.159668922424
5


KeyboardInterrupt: 

In [81]:
model.predict(204079534)

[tensor(1913), tensor(364), tensor(127)]

In [ ]:
test_data.pav_order_id'

In [80]:
hist_data.shape

(4529889, 8)

### Prepare Submission File

In [100]:
from models.hmodel import HModel

model = HModel(top_n = 5, seed = 32)
model.train(hist_data)

In [101]:
model.shash

tensor([[0.0000, 0.0000, 0.7368,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [109]:
z = model.predict(test_data)

KeyboardInterrupt: 

In [ ]:
z

In [108]:
from collections import Counter

c = Counter()
c.update([1])
c

Counter({1: 1})

In [32]:
model.shash

item_id
202782406    [(20, 95), (13, 78), (4, 74), (26, 73), (2, 70)]
202782407        [(20, 2), (16, 2), (15, 1), (5, 1), (10, 1)]
202782412        [(7, 6), (13, 4), (20, 4), (21, 3), (17, 2)]
202782414       [(13, 10), (24, 9), (28, 9), (11, 9), (9, 9)]
202782415     [(2, 21), (31, 21), (6, 20), (7, 18), (17, 18)]
                                   ...                       
218175579          [(19, 3), (8, 2), (9, 2), (20, 1), (2, 1)]
218199321        [(13, 4), (17, 3), (10, 3), (7, 2), (11, 2)]
218217955        [(14, 4), (27, 3), (11, 2), (23, 2), (4, 2)]
218217991          [(14, 2), (1, 2), (4, 1), (28, 1), (6, 1)]
218240709         [(28, 2), (3, 2), (19, 2), (7, 2), (17, 1)]
Name: sid_left, Length: 52472, dtype: object

In [34]:
norm = torch.zeros(32)
for s, v in model.norm.iteritems():
    norm[s] = v

In [35]:
norm




tensor([166497., 161872., 137723., 141669., 160600., 131611., 120247., 154120.,
        145422., 114079., 120855., 130205., 122878., 204036.,  86697., 137561.,
        135003., 152091., 101188., 147692., 224414., 148674., 136301., 121300.,
        147632., 145180., 127464., 137618., 166937., 120814., 127741., 153768.])

In [50]:
import torch

def to_vec(x, seed=32):
    vec = torch.zeros(seed)
    for c,w in x:
        vec[c] = w
    return vec.reshape(1, -1)

t = model.shash.apply(to_vec)
X = torch.cat([*t.values], axis=0)
X_norm = X.max(axis=1)

X = X / X_norm.values.reshape(-1,1)

def idx(X0, x):
    idx = torch.argmin(
        torch.sum(
            torch.abs(X - X[0, :].reshape(1,-1)), axis=1), axis=0)
    return idx

dist(X, X[0, :])

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

tensor(0)

In [69]:
X - X[0, :].reshape(1,-1)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.7368,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.7368,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000, -0.7368,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  1.0000, -0.7368,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.7368,  ...,  0.0000,  0.0000,  0.0000]])

In [63]:
X[0,:]

tensor([0.0000, 0.0000, 0.7368, 0.0000, 0.7789, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.8211, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7684,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000])

In [64]:
X[14,:]

tensor([0.0000, 0.0000, 0.0000, 0.8333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.6667, 0.8333, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000])

In [59]:
x = X[0, :]
x

tensor([0.0000, 0.0000, 0.7368, 0.0000, 0.7789, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.8211, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7684,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000])

In [37]:
X_norm.values

tensor([51059., 48803., 15659.,  9159., 49218.,  2623.,  4295., 15044., 11521.,
         1882.,  1734.,  3158.,  9008., 65631.,  2058.,  1801.,  2702.,  9947.,
          789.,  5894., 47312., 31467.,  2300.,  2912., 10992.,  3272.,  2299.,
         3754., 53616.,  1736.,  2220.,  7640.])

In [14]:
x0 = X[0, :]
x0

tensor([ 0.,  0., 70.,  0., 74.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 78.,
         0.,  0.,  0.,  0.,  0.,  0., 95.,  0.,  0.,  0.,  0.,  0., 73.,  0.,
         0.,  0.,  0.,  0.])

In [15]:
xy = X @ x0
xx = x0.T @ x0
yy = X @ X.T

In [16]:
xy.shape, xx.shape, yy.shape

(torch.Size([52472]), torch.Size([]), torch.Size([52472, 52472]))

In [21]:
torch.linalg.pinv(xx * yy)


Intel MKL ERROR: Parameter 12 was incorrect on entry to SGESDD.


RuntimeError: falseINTERNAL ASSERT FAILED at "../aten/src/ATen/native/LinearAlgebraUtils.h":244, please report a bug to PyTorch. svd_cpu: Argument 12 has illegal value. Most certainly there is a bug in the implementation calling the backend library.

In [36]:
X_norm = X.max(axis=0).values

In [40]:
X[:, 0].max()

tensor(51059.)

In [34]:
X_norm

tensor([95.,  2.,  6.,  ...,  4.,  2.,  2.])

In [24]:
model.shash.iloc[0]

[(20, 95), (13, 78), (4, 74), (26, 73), (2, 70)]

In [5]:
hist_data

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods,weight,sid
0,95203091,98506637863,2021-07-01 00:03:44,202808329,1.000,79.990,False,11.14,9
1,95203091,98506637863,2021-07-01 00:03:44,202953905,1.072,44.945,True,11.14,17
2,95203091,98506637863,2021-07-01 00:03:44,203566452,1.000,69.990,False,11.14,20
3,95203091,98506637863,2021-07-01 00:03:44,202820143,1.972,41.295,True,11.14,15
4,95203091,98506637863,2021-07-01 00:03:44,204400422,1.000,269.990,False,11.14,6
...,...,...,...,...,...,...,...,...,...
4529884,95619205,4620221347,2021-09-07 18:48:29,204351817,1.000,99.990,False,9.84,9
4529885,95619205,4620221347,2021-09-07 18:48:29,203523604,1.000,62.190,False,9.84,20
4529886,95619205,4620221347,2021-09-07 18:48:29,203566357,2.000,39.890,False,9.84,21
4529887,95619205,4620221347,2021-09-07 18:48:29,203566418,1.000,62.190,False,9.84,18


In [3]:
test = model.predict(test_data)
test['preds'].to_csv('pred.csv')

### Test Model on Holdout

#### Prepare Splits

In [4]:

from models.core import split_data
from models.baseline import BaselineModel


# разобьем историю в отношении 70 на 30 для трейна и валидации
train_data, test_data, orders_sort, train_orders, test_orders = split_data(hist_data)



#### Play with top_n hyperparameter of baseline model

In [5]:
model = BaselineModel(top_n = 10)
model.train(train_data)
model.quality(test_data)

0.34481132996828945

In [6]:
model = BaselineModel(top_n = 15)
model.train(train_data)
model.quality(test_data)

0.3146923962889297

In [7]:
model = BaselineModel(top_n = 5)
model.train(train_data)
model.quality(test_data)

0.39452886611806376